<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/multi_agent_comm_via_mcp/multi_agent_mcp_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi‑Agent Supervisor Design with MCP‑style Messaging (Notebook)

This notebook shows a **supervisor agent** orchestrating two **worker agents** using a lightweight, in-notebook simulation of the **Model Context Protocol (MCP)** message pattern.

You'll be able to run everything end‑to‑end here (no network access needed). In the final section, you'll find a **drop‑in appendix** with real MCP SDK starter snippets so you can swap the in‑memory transport for actual MCP **clients/servers** on your machine.

### Why simulate?
Real MCP involves a JSON‑RPC protocol over transports like stdio / SSE / HTTP. Implementing and spawning separate processes in a teaching notebook gets fiddly, so we provide a faithful **message shape** and **supervisor routing logic** that mirrors MCP calls and results. Then, when you're ready, you can move to real servers with the provided code stubs.

> **References**  
> • Model Context Protocol overview and docs  
> • Python SDK for building MCP servers/clients  
> • OpenAI docs on MCP within the Agents SDK  

---
## Architecture

```
+-------------------+          JSON-RPC-ish messages          +------------------+
|   Supervisor      |  ------------------------------------>  |  Worker A (Math) |
|   Agent           |  <------------------------------------  |  Tools: add, avg |
+---------+---------+                                          +------------------+
          |                                                                
          |                                  +------------------+
          |--------------------------------->|  Worker B (Text) |
          |<---------------------------------|  Tools: upper, kw|
                                             +------------------+
```

The **Supervisor** receives a user goal, decomposes it to sub‑tasks, sends RPC‑style `call_tool` requests to workers, and fuses the results.


In [ ]:
# Core types and a tiny in-memory MCP-style bus
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Any, Dict, Optional, Callable, List
import uuid


# --- MCP(ish) message shapes (subset of JSON-RPC + MCP semantics) ---
@dataclass
class MCPRequest:
    jsonrpc: str = "2.0"
    method: str = "tools/call"  # e.g., tools/call
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    params: Dict[str, Any] = field(default_factory=dict)

@dataclass
class MCPResponse:
    jsonrpc: str = "2.0"
    id: Optional[str] = None
    result: Any = None
    error: Optional[Dict[str, Any]] = None


class MCPBus:
    """A minimal in-memory router that mimics an MCP transport.
    Agents register a name and a handler. Requests are routed by 'target' in params.
    """
    def __init__(self):
        self._handlers: Dict[str, Callable[[MCPRequest], MCPResponse]] = {}

    def register(self, name: str, handler: Callable[[MCPRequest], MCPResponse]):
        if name in self._handlers:
            raise ValueError(f"Handler {name} already registered")
        self._handlers[name] = handler

    def send(self, req: MCPRequest) -> MCPResponse:
        target = req.params.get("target")
        if target not in self._handlers:
            return MCPResponse(id=req.id, error={"code": -32601, "message": f"Unknown target: {target}"})
        try:
            return self._handlers[target](req)
        except Exception as e:
            return MCPResponse(id=req.id, error={"code": -32000, "message": str(e)})


In [ ]:
# Agent base class and two workers exposing MCP-style tools
class BaseAgent:
    def __init__(self, name: str, bus: MCPBus):
        self.name = name
        self.bus = bus

    # For workers: handle a single MCPRequest and return MCPResponse
    def handle(self, req: MCPRequest) -> MCPResponse:
        raise NotImplementedError


class MathWorker(BaseAgent):
    """Exposes tools: add(numbers), avg(numbers)"""
    def handle(self, req: MCPRequest) -> MCPResponse:
        # Expect: params = { target: 'math', tool: 'add'|'avg', args: {...} }
        tool = req.params.get("tool")
        args = req.params.get("args", {})
        if tool == "add":
            nums = args.get("numbers", [])
            result = sum(nums)
        elif tool == "avg":
            nums = args.get("numbers", [])
            result = (sum(nums) / len(nums)) if nums else 0
        else:
            return MCPResponse(id=req.id, error={"code": -32601, "message": f"Unknown tool: {tool}"})
        return MCPResponse(id=req.id, result={"ok": True, "tool": tool, "value": result})


class TextWorker(BaseAgent):
    """Exposes tools: upper(text), keywords(text, k=5)"""
    def handle(self, req: MCPRequest) -> MCPResponse:
        tool = req.params.get("tool")
        args = req.params.get("args", {})
        if tool == "upper":
            text = args.get("text", "")
            result = text.upper()
        elif tool == "keywords":
            text = args.get("text", "")
            k = int(args.get("k", 5))
            # A toy keyword extractor: take unique words by frequency/length
            import re
            words = [w.lower() for w in re.findall(r"[A-Za-z0-9']+", text)]
            from collections import Counter
            common = Counter(words)
            # sort by (count desc, length desc)
            kws = sorted(common.items(), key=lambda x: (x[1], len(x[0])), reverse=True)
            result = [w for w, _ in kws[:k]]
        else:
            return MCPResponse(id=req.id, error={"code": -32601, "message": f"Unknown tool: {tool}"})
        return MCPResponse(id=req.id, result={"ok": True, "tool": tool, "value": result})


In [ ]:
# Supervisor agent: plans, dispatches to workers via the MCPBus, and fuses results
class Supervisor(BaseAgent):
    def __init__(self, name: str, bus: MCPBus):
        super().__init__(name, bus)
        self.log: List[str] = []

    def plan(self, user_goal: str) -> List[Dict[str, Any]]:
        """Very small heuristic planner that looks for numbers and text in the goal."""
        import re
        numbers = [float(x) for x in re.findall(r"-?\d+(?:\.\d+)?", user_goal)]
        wants_avg = any(w in user_goal.lower() for w in ["average", "avg", "mean"])
        wants_sum = any(w in user_goal.lower() for w in ["sum", "total"]) and not wants_avg
        wants_upper = "uppercase" in user_goal.lower() or "upper" in user_goal.lower()
        wants_keywords = "keywords" in user_goal.lower()

        tasks = []
        if numbers and wants_avg:
            tasks.append({"target": "math", "tool": "avg", "args": {"numbers": numbers}})
        elif numbers and wants_sum:
            tasks.append({"target": "math", "tool": "add", "args": {"numbers": numbers}})
        # Derive text from quotes as a toy signal
        import re
        quotes = re.findall(r'"([^"]+)"|\'([^\']+)\'', user_goal)
        text_segments = [q[0] or q[1] for q in quotes]
        if text_segments:
            text = " ".join(text_segments)
            if wants_upper:
                tasks.append({"target": "text", "tool": "upper", "args": {"text": text}})
            if wants_keywords:
                tasks.append({"target": "text", "tool": "keywords", "args": {"text": text, "k": 5}})
        return tasks

    def call_tool(self, target: str, tool: str, args: Dict[str, Any]) -> Dict[str, Any]:
        req = MCPRequest(params={"target": target, "tool": tool, "args": args})
        self.log.append(f"→ call {target}.{tool}({args}) [id={req.id}]")
        resp = self.bus.send(req)
        if resp.error:
            self.log.append(f"← ERROR {target}.{tool}: {resp.error}")
            return {"ok": False, "error": resp.error}
        self.log.append(f"← result {target}.{tool}: {resp.result}")
        return resp.result

    def handle_user_goal(self, goal: str) -> Dict[str, Any]:
        self.log.clear()
        self.log.append(f"User goal: {goal}")
        plan = self.plan(goal)
        self.log.append(f"Plan: {plan}")
        results = []
        for step in plan:
            results.append(self.call_tool(step["target"], step["tool"], step["args"]))
        # Simple fusion: collect each result value in a dict keyed by tool
        fused: Dict[str, Any] = {}
        for r in results:
            if r.get("ok"):
                fused[r["tool"]] = r["value"]
        return {"plan": plan, "results": results, "fused": fused, "log": self.log[:]}


In [ ]:
# Wire it all together and run a demo
bus = MCPBus()
math_worker = MathWorker("math", bus)
text_worker = TextWorker("text", bus)
supervisor = Supervisor("supervisor", bus)

bus.register("math", math_worker.handle)
bus.register("text", text_worker.handle)

goal = (
    'Compute the average of 10, 20, 30 and also extract keywords and uppercase the phrase "Agents coordinate via MCP messages".'
)
result = supervisor.handle_user_goal(goal)
result

In [ ]:
# Pretty-print the supervisor log
for line in result["log"]:
    print(line)

## Try your own goal
Change the text below and re-run the cell to see different plans and tool calls.

In [ ]:
your_goal = 'Sum 5, 12, 13 and uppercase the quote "mcp makes tools reusable".'
supervisor.handle_user_goal(your_goal)

---
## Appendix: Swap the in-memory bus for **real MCP**

Below are annotated starter snippets. Run these **outside** this hosted environment (e.g., locally) with internet access.

### 1) Install the Python MCP SDK
```bash
pip install mcp anyio
```

### 2) Minimal MCP Server (Worker)
```python
import anyio
from mcp.server import Server
from mcp.types import Tool

server = Server(name="math")

@server.tool()
async def add(numbers: list[float]) -> float:
    return float(sum(numbers))

@server.tool()
async def avg(numbers: list[float]) -> float:
    return float(sum(numbers)/len(numbers))

if __name__ == "__main__":
    # Run as stdio transport so MCP clients can spawn this process
    anyio.run(server.run_stdio)
```

Create another server for `text` with tools `upper(text: str) -> str` and `keywords(text: str, k: int = 5) -> list[str]`.

### 3) Minimal MCP Client (Supervisor)
```python
import anyio
from mcp.client.session import Session
from mcp.client.stdio import StdioServerParameters, stdio_client

async def main():
    # Launch two worker servers as child processes (stdio transport)
    math_params = StdioServerParameters(command=["python", "math_server.py"])  # path to your file
    text_params = StdioServerParameters(command=["python", "text_server.py"])  

    async with (
        stdio_client(math_params) as math_conn,
        stdio_client(text_params) as text_conn,
        Session(math_conn) as math_sess,
        Session(text_conn) as text_sess,
    ):
        # Discover tools (optional)
        await math_sess.initialize()
        await text_sess.initialize()

        # Call tools
        add_res = await math_sess.call_tool("add", {"numbers": [10,20,30]})
        upper_res = await text_sess.call_tool("upper", {"text": "mcp client/server demo"})
        print(add_res, upper_res)

anyio.run(main)
```

In a full application, you'd embed planning logic (like the `Supervisor` earlier) and call the appropriate MCP server tool based on the task decomposition.

### 4) Security Tip
Only run **trusted** MCP servers and review their permissions. Treat them like any executable plugin.

— End of Notebook —